In [22]:
import tensorflow as tf
from keras.src.models import Sequential
from keras.src.layers import Dense
from keras.src.callbacks import TensorBoard
from keras.api.models import save_model, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import os
import pandas as pd

In [23]:
data_path = "../data"
csv_data = f"{data_path}/data.csv"
filtered_data = f"{data_path}/filtered.csv"
segment_path = f"{data_path}/segments"
segment_data = segment_path + "/segment_{}.csv"

In [24]:
headers = {
    "RequestID": str,
    "Boro": str,
    "Yr": str,
    "M": str,
    "D": str,
    "HH": str,
    "MM": str,
    "Vol": str,
    "SegmentID": str,
    "WktGeom": str,
    "street": str,
    "fromSt": str,
    "toSt": str,
    "Direction": str
}

garbage_headers = [
    "RequestID",
    "Boro",
    "WktGeom",
    "street",
    "fromSt",
    "toSt",
    "Direction"
]

needed_headers = {
    "Yr": int,
    "M": int,
    "D": int,
    "HH": int,
    "MM": int,
    "Vol": int,
    "SegmentID": int,
}

if not os.path.exists(filtered_data):
    os.mkdir(data_path) if not os.path.exists(data_path) else ...
    ds = pd.read_csv(csv_data, header=None, names=list(headers.keys()), dtype=headers)
    for h in garbage_headers:
        ds.drop(columns=h, inplace=True)
    ds.drop(ds.index[0], axis="index", inplace=True)
    ds.to_csv(filtered_data, header=None, index=None)

In [25]:
ds = pd.read_csv(filtered_data, header=None, names=list(needed_headers.keys()), dtype=needed_headers)

In [26]:
if not os.path.exists(segment_path):
    os.mkdir(segment_path)
    for segmentID in ds["SegmentID"].unique():
        ds.loc[ds["SegmentID"] == segmentID].drop(columns="SegmentID").to_csv(segment_data.format(segmentID))

In [27]:
del needed_headers["SegmentID"]

segmentID = 83624  # Largest Segment

ds = pd.read_csv(segment_data.format(segmentID), header=None, names=list(needed_headers.keys()), dtype=needed_headers)